### Problem 18
Given an ideal experimental spectrum, find a cyclic peptide whose theoretical spectrum matches the experimental spectrum.

Given: A collection of (possibly repeated) integers Spectrum corresponding to an ideal experimental spectrum.

Return: Every amino acid string Peptide such that Cyclospectrum(Peptide) = Spectrum (if such a string exists).

Functions:
- Expand(Peptides)：a new collection containing all possible extentions of peptides in Peptides by a single amino acid.
- ParentMass(Spectrum): the peptide’s mass in mass spectrometry experiments
- Mass(Peptide): the largest mass in the spectrum.

Algorithm:

    CYCLOPEPTIDESEQUENCING(Spectrum):
        Peptides ← a set containing only the empty peptide
            while Peptides is nonempty
                Peptides ← Expand(Peptides)
                for each peptide Peptide in Peptides
                    if Mass(Peptide) = ParentMass(Spectrum)
                        if Cyclospectrum(Peptide) = Spectrum
                            output Peptide
                        remove Peptide from Peptides
                    else if Peptide is not consistent with Spectrum
                        remove Peptide from Peptides

In [144]:
class generatePeptide():
    """
    A class to produce cyclic peptides whose theoretical spectrum matches the experimental spectrum

    ...

    Attributes
    ----------
    experimentSpec : list
        experimental spectrum
    masses : list
        the masses of standard amino acid

    Methods
    -------
    expand(Peptides):
        all extentions of peptides whose total mass in experimental spectrum by a single amino acid
    Cyclospectrum(Peptides):
        generate Spectrum for the cyclic peptide
    cycloPeptideSequencing():
        return the Peptides with identical spectrum to the experimental spectrum
    """
    def __init__(self,infile):  
        '''
        contructor: saves attributes 
        
        Parameters
        ----------
            infile : file name
                
        '''
        with open(infile) as file:
            #turn the element of experiment spectrum to int and sort the list
            self.experimentSpec=sorted([int(i) for i in file.readline().strip().split()]) 
        #this is all the mass of standard amino acid
        masses=[57, 71, 87, 97, 99, 101, 103, 113, 114, 115, 128, 129, 131, 137, 147, 156, 163, 186]
        #filter the masses and remain those are consistent to experiment spectrum
        self.masses=[i for i in masses if i in self.experimentSpec]
    
    def expand(self,Peptides):
        """
        Generate the SubPeptide for a peptide
        
        Parameters
        ----------
        Peptides:list
            The current peptides
        
        Return
        ----------
        Peptides:list
            New peptides with one amino acid extended
        """
        #----------------copy every peptide for 18 times so that each of them extend an amino acid--------------#
        for i in list(Peptides):
            j=1
            while j<len(self.masses): #keep copying every element for 18 times
                Peptides.append(list(i)) #the reason I use list(i) is that to avoid append an element with same id
                j+=1
        Peptides.sort() #sort the Peptides to make sure the same copies of element are put together
        #----------------copy every peptide for 18 times so that each of them extend an amino acid--------------#
        
        #--------------------------here we extend 18 amino acids to each of the subpeptides---------------------#
        removePep=[]
        for start in range(0,len(Peptides),len(self.masses)):#every 18 elements(len(self.masses)) in peptides are in a group
            for i in range(start,start+len(self.masses)): #start:start position for a new group
        #------we append those inconsistent next amino acid to removePep and then remove them from Peptides-----#
                temp=Peptides[i][:] #here we assign to next amino acid to extend to temp
                temp.append(self.masses[i%len(self.masses)]) #temp is the peptide with the amino acid we're going to extend
                if sum(temp) not in self.experimentSpec:#if the sum of masses in temp not in self.experimentSpec
                    removePep.append(Peptides[i])
                    continue
                else: #temp is consistent
                    Peptides[i].append(self.masses[i%len(self.masses)])
        #------we append those inconsistent next amino acid to removePep and then remove them from Peptides-----#
        
        #--------------------------here we extend 18 amino acids to each of the subpeptides---------------------#
        for i in removePep:#we are removing those inconsistent peptide from Peptides
            Peptides.remove(i)
        return Peptides
    
    def Cyclospectrum(self,Peptides):
        """
        Generate the masses of subpeptides 
        
        Parameters
        ----------
        Peptides:list
            The current peptides
        
        Return
        ----------
        sorted(subPeptides):list
            the sorted masses of Peptides
        """
        Peptides.remove(0) #when we simulate the cyclic peptide, we first remove 0
        cicPeptide=list(Peptides) #avoid the Peptides and cicPeptide share the same id
        cicPeptide.extend(Peptides[:len(Peptides)-2]) #to mimic a cyclic peptide, we entend some amino acids to the end
        subPeptides=[] #save all the subPeptides
        for i in range(1,len(Peptides)): #generate subpeptides for all length
            for j in range(len(Peptides)): #just like generate kmer, j is the start position
                subPeptides.append(sum(cicPeptide[j:j+i]))
        subPeptides.extend([0,sum(Peptides)]) #we finally add the complete peptide to the list
        return sorted(subPeptides) 
    
    def cycloPeptideSequencing(self):
        """
        Produce cyclic peptides whose theoretical spectrum matches the experimental spectrum

        Return
        ----------
        candidates:list
            the possible cyclic peptides
        """
        Peptides=[[0]] #Peptides start with 0
        candidates=[] #this is a list saving peptides whose spectrum identical to the experimental one
        while Peptides: # if the Peptides not empty
            Peptides=self.expand(Peptides) #call the expand function to expand amino acid
            for i in list(Peptides): 
                if sum(i)==max(self.experimentSpec): #if Mass(Peptide) = ParentMass(Spectrum)
                    if self.Cyclospectrum(i)==self.experimentSpec: #if the spectrums are the same
                        candidates.append('-'.join(map(str,i)))# format the output
                        Peptides.remove(i)
        return candidates

### Main

In [147]:
def main(infile):
    '''
    Find cyclic peptides here
    
    Parameters
        ----------
        infile : str 
            the filename of the fasta file

        Returns
        -------
        STDOUT
    '''
    Peptide=generatePeptide(infile) #instantiation
    Mass=Peptide.cycloPeptideSequencing()
    with open('output.txt','w') as file:
        for i in Mass:
            print(i,end='\t',file=file)

### Run the program here

In [149]:
if __name__ == "__main__":
    main('rosalind_ba4e.txt')